# 4---เริ่มทำการตรวจจับคำ trigger word detection

In [2]:
import numpy as np
import time
from pydub import AudioSegment
import random
import sys
import io 
import os
import glob
import IPython
import tensorflow as tf
from tensorflow import keras
import wave
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
%matplotlib inline

## Load model

In [3]:
model_help = load_model('C:/Users/sawalee/Desktop/detect_Help/Code_Test/models/help_model_new8.h5')
model_no = load_model('C:/Users/sawalee/Desktop/detect_Help/Code_Test/models/No_11.h5')
model_yes = load_model('C:/Users/sawalee/Desktop/detect_Help/Code_Test/models/Yes_model_6.h5')

## ------------------------------------------------------------------------------

## Detect trigger word Help
##### โหลดโมเดลมาไว้ใน Func- detect_triggerword_spectrum เพื่อเอาไปเช็คใน Audio stream เมื่อเจอคำว่า "ช่วยด้วย" แสดง "Do you need help? "

In [4]:
def detect_triggerword_spectrum_help(x):

    x  = x.swapaxes(0,1)
    x = np.expand_dims(x, axis=0)
    predictions = model_help.predict(x)
    return predictions.reshape(-1)

## Detect trigger word No
##### โหลดโมเดลมาไว้ใน Func- detect_triggerword_spectrum เพื่อเอาไปเช็คใน Audio stream เมื่อเจอคำว่า "ไม่" แสดง " No "

In [5]:
def detect_triggerword_spectrum_no(x):

    x  = x.swapaxes(0,1)
    x = np.expand_dims(x, axis=0)
    predictions = model_no.predict(x)
    return predictions.reshape(-1)

## Detect trigger word Yes
##### โหลดโมเดลมาไว้ใน Func- detect_triggerword_spectrum เพื่อเอาไปเช็คใน Audio stream เมื่อเจอคำว่า "ใช่" แสดง "Sending a message for help"

In [6]:
def detect_triggerword_spectrum_yes(x):

    x  = x.swapaxes(0,1)
    x = np.expand_dims(x, axis=0)
    predictions = model_yes.predict(x)
    return predictions.reshape(-1)

## ------------------------------------------------------------------------------

In [7]:
def has_new_triggerword_help(predictions, chunk_duration, feed_duration, threshold=0.01):  #threshold สามารถปรับเปลี่ยนได้

    predictions = predictions > threshold
    
    chunk_predictions_samples = int(len(predictions) * chunk_duration / feed_duration)
    chunk_predictions = predictions[-chunk_predictions_samples:]
    level = chunk_predictions[0]
    for predictions in chunk_predictions:
        if predictions > level:
            return True
        else:
            level = predictions
    return False

def has_new_triggerword_no(predictions, chunk_duration, feed_duration, threshold=0.01):  #threshold สามารถปรับเปลี่ยนได้

    predictions = predictions > threshold
    
    chunk_predictions_samples = int(len(predictions) * chunk_duration / feed_duration)
    chunk_predictions = predictions[-chunk_predictions_samples:]
    level = chunk_predictions[0]
    for predictions in chunk_predictions:
        if predictions > level:
            return True
        else:
            level = predictions
    return False

def has_new_triggerword_yes(predictions, chunk_duration, feed_duration, threshold=0.5):  #threshold สามารถปรับเปลี่ยนได้

    predictions = predictions > threshold
    
    chunk_predictions_samples = int(len(predictions) * chunk_duration / feed_duration)
    chunk_predictions = predictions[-chunk_predictions_samples:]
    level = chunk_predictions[0]
    for predictions in chunk_predictions:
        if predictions > level:
            return True
        else:
            level = predictions
    return False

## Record audio stream from mic 

In [8]:
chunk_duration = 0.5
fs = 44100 
chunk_samples = int(fs * chunk_duration)

feed_duration = 10
feed_samples = int(fs * feed_duration)

assert feed_duration/chunk_duration == int(feed_duration/chunk_duration)

In [9]:
def get_spectrogram(data):

    nfft = 200 
    fs = 8000 
    noverlap = 120
    nchannels = data.ndim
    if nchannels == 1:
        pxx, _, _ = mlab.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, _, _ = mlab.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

In [10]:
def plt_spectrogram(data):

    nfft = 200
    fs = 8000
    noverlap = 120 
    nchannels = data.ndim
    if nchannels == 1:
        pxx, _, _, _ = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, _, _, _ = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

In [11]:
def get_audio_input_stream(callback):
    stream = pyaudio.PyAudio().open(
        format=pyaudio.paInt16,
        channels=1,
        rate=fs,
        input=True,
        frames_per_buffer=chunk_samples,
        input_device_index=1,
        stream_callback=callback)
    return stream

## ------------------------------------------------------------------------------
# Start stream

In [ ]:
import pyaudio
from queue import Queue
from threading import Thread
import sys
import time
from playsound import playsound
from multiprocessing import Queue  # multiprocessing queue

q = Queue()
run = True
silence_threshold = 400

data = np.zeros(feed_samples, dtype='int16')

def callback(in_data, frame_count, time_info, status):
    global run, timeout, data, silence_threshold          
    data0 = np.frombuffer(in_data, dtype='int16')
    if np.abs(data0).mean() < silence_threshold:
        sys.stdout.write('-')
        return (in_data, pyaudio.paContinue)
    else:
        sys.stdout.write('.')
    data = np.append(data,data0)    
    if len(data) > feed_samples:
        data = data[-feed_samples:]
        q.put(data)
    return (in_data, pyaudio.paContinue)

stream = get_audio_input_stream(callback)
stream.start_stream()
   
try:
    while run:                
        data = q.get()
        spectrum = get_spectrogram(data) 
        print("Listening Help")
        preds_help = detect_triggerword_spectrum_help(spectrum)        
        new_trigger_help = has_new_triggerword_help(preds_help, chunk_duration, feed_duration)  
        
#         print("Check Help")
        if new_trigger_help:
            print('Do you need help?')
            playsound('output3.wav') 
            print('Yes / No')
            playsound('output5.wav')
            
            timeCount = 0
            maxCount = 10
            
            while timeCount < maxCount:
                timeCount += 1
#                 print(timeCount)
                
                data = q.get()
                spectrum = get_spectrogram(data) 
                
                preds_no = detect_triggerword_spectrum_no(spectrum)
                preds_yes = detect_triggerword_spectrum_yes(spectrum)
                new_trigger_no = has_new_triggerword_no(preds_no, chunk_duration, feed_duration)
                new_trigger_yes = has_new_triggerword_yes(preds_yes, chunk_duration, feed_duration)
                
#                 print("Check yes")
                if new_trigger_yes:
                    print('Sending a message for help')
                    playsound('output4.wav')
                    time.sleep(3)
                    break;
                
#                 print("Check No")
                if new_trigger_no:
                    print('No')
                    playsound('output6.wav')
                    time.sleep(3)
                    break; 
                    
#             print("Listening Help")
      
    print("End")
    
except (KeyboardInterrupt, SystemExit):
    stream.stop_stream()
    stream.close()

-------.Listening Help
.---------..----------------------------..----Listening Help
Listening Help
-Listening Help
-Do you need help?
-----Yes / No
----------------.-No
---------------------.Listening Help
.Listening Help
-------------.Listening Help
.Listening Help
Do you need help?
------Yes / No
--------------.-

## Stop stream

### Kernel --> Restart kernel... เพื่อหยุด stream